In [2]:
#Importing libraries
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

**Data Cleaning and Preprocessing**

Lets first load our dataset and have a first look. This will help us decide what kind of transformations and data cleaning steps we should do.

In [3]:
# --- Load Dataset ---
df = pd.read_csv("../src/data/used_cars.csv")
df.head(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"
5,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
8,Lexus,RC 350 F Sport,2021,"23,436 mi.",Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,None reported,Yes,"$41,927"
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"


To prepare the dataset for analysis, we first cleane key numeric columns. The price and milage fields are originally strings containing symbols like $, ,, and "mi.", which we will strip out to convert them into usable numerical formats.

We will then handle binary fields:

accident is converted to 0 (no accident) or 1 (accident reported), treating missing values as no accidents.

clean_title is also binarized: 1 for clean titles and 0 otherwise, including missing values.

Finally, rows missing fuel_type are dropped to maintain data consistency for a critical categorical feature.

In [4]:
# --- Clean Price & Mileage ---
df["price"] = df["price"].replace(r'[\$,]', '', regex=True).astype(float)
df["milage"] = df["milage"].replace(r'[,mi.]', '', regex=True).astype(float)

# --- Binary Conversion ---
df["accident"] = df["accident"].apply(lambda x: 0 if pd.isna(x) or "None reported" in str(x) else 1)
df["clean_title"] = df["clean_title"].apply(lambda x: 1 if x == "Yes" else 0)
df["clean_title"] = df["clean_title"].fillna(0)
df = df.dropna(subset=["fuel_type"]).copy()

*Feature Engineering*

Next, I will extract additional features from the unstructured text columns to enhance the dataset. From the engine column, I will derive engine_size, cylinders, horsepower (hp), fuel injection type (fi_type), and whether the car is turbocharged or not. From the transmission column, I will extract the trans_type and number of trans_speed.

I will also extract clean versions of the exterior and interior colors, along with whether the paint is metallic or gloss, from the ext_col and int_col fields. These engineered features will help improve model performance and support deeper analysis.

In [5]:
# --- Feature Extraction Functions ---
def get_hp(spec):
    return float(re.search(r'(\d+(?:\.\d+)?)HP', spec).group(1)) if re.search(r'(\d+(?:\.\d+)?)HP', spec) else pd.NA

def get_engine(spec):
    return float(re.search(r'(\d+(?:\.\d+)?)\s?L(iter)?', spec).group(1)) if re.search(r'(\d+(?:\.\d+)?)\s?L(iter)?', spec) else pd.NA

def get_cylinder(spec):
    match = re.search(r'(?i)(V\d+|i-?\d+|Flat\s+\d+|Straight\s+\d+|\d+ Cylinder)', spec)
    return match.group(1).upper() if match else 'NA'

def get_fi(spec):
    match = re.search(r'(?i)(PDI|GDI|MPFI|TFSI|DDI|SIDI|GTDI|TSI)', spec)
    return match.group(0) if match else 'NA'

def get_turbo(spec):
    match = re.search(r'(?i)(Turbo|Twin Turbo|Supercharged|T/C)', spec)
    return match.group(0) if match else 'NA'

def get_trans_type(spec):
    match = re.search(r'(?i)(Automatic cvt|Automatic|CVT|manual|M/T|A/T|At|Mt)', spec)
    return match.group(0).upper() if match else 'M/T'

def get_speed(spec):
    match = re.search(r'(?i)(\d+)-?\s?(?:speed)?', spec)
    return int(match.group(1)) if match else pd.NA

def get_color(spec):
    match = re.search(r'(?i)(Black|White|Gray|Silver|Blue|Red|Green|Gold|Brown|Orange|Beige|Yellow|Ebony)', spec)
    return match.group(0) if match else 'Miscellaneous'

def get_color_type(spec):
    match = re.search(r'(?i)(Metallic|Metal|chrome)', spec)
    return match.group(0) if match else 'Gloss'

# --- Apply Feature Engineering ---
df["engine_size"] = df["engine"].apply(get_engine)
df["cylinders"] = df["engine"].apply(get_cylinder)
df["hp"] = df["engine"].apply(get_hp)
df["fi_type"] = df["engine"].apply(get_fi)
df["turbo"] = df["engine"].apply(get_turbo)
df["trans_type"] = df["transmission"].apply(get_trans_type)
df["trans_speed"] = df["transmission"].apply(get_speed)
df["ext_base_color"] = df["ext_col"].apply(get_color)
df["ext_color_type"] = df["ext_col"].apply(get_color_type)
df["int_base_color"] = df["int_col"].apply(get_color)


Next, I normalize the trans_type values because the transmission types appear in multiple inconsistent formats (e.g., "A/T", "At", "Automatic CVT"). To avoid treating these as separate categories during modeling, I map all variations to clean, consistent labels like "Automatic", "Manual", and "Variable".

Then, I ensure that the hp, engine_size, and trans_speed columns are properly converted to numeric types. This step is important to prevent issues during modeling or statistical calculations, especially after applying transformations from string-based features.

After that, I handle missing values in key numerical features using group-wise median imputation. Instead of using a global median, I group data by relevant categorical variables (cylinders or trans_type) to preserve group-level patterns.

For hp and engine_size, I use the median of the respective cylinders group.

For trans_speed, I use the median of the trans_type group.

This ensures more context-aware and realistic filling of missing data, reducing potential bias or noise in model input.

In [6]:
# --- Normalize Transmission Labels ---

df['trans_type'] = df['trans_type'].replace({
    'A/T': 'Automatic',
    'At': 'Automatic',
    'A/t': 'Automatic',
    'AT': 'Automatic',
    'M/T': 'Manual',
    'Mt': 'Manual',
    'M/t': 'Manual',
    'MT': 'Manual',
    'MANUAL': 'Manual',
    'CVT': 'Variable',
    'AUTOMATIC': 'Automatic',
    'Automatic Cvt': 'Variable'
})

# Ensure correct types
df["hp"] = pd.to_numeric(df["hp"], errors="coerce")
df["engine_size"] = pd.to_numeric(df["engine_size"], errors="coerce")
df["trans_speed"] = pd.to_numeric(df["trans_speed"], errors="coerce")

# --- Median Imputation (Corrected with Group Filters) ---
# HP
valid_hp = df[df['hp'].notna() & (df['cylinders'] != 'NA')]
hp_group_median = valid_hp.groupby('cylinders')['hp'].median()
df['hp'] = df.apply(lambda row: hp_group_median.get(row['cylinders'], df['hp'].median()) if pd.isna(row['hp']) else row['hp'], axis=1)

# Engine Size
valid_engine = df[df['engine_size'].notna() & (df['cylinders'] != 'NA')]
engine_group_median = valid_engine.groupby('cylinders')['engine_size'].median()
df['engine_size'] = df.apply(lambda row: engine_group_median.get(row['cylinders'], df['engine_size'].median()) if pd.isna(row['engine_size']) else row['engine_size'], axis=1)

# Transmission Speed
valid_speed = df[df['trans_speed'].notna() & df['trans_type'].notna()]
speed_group_median = valid_speed.groupby('trans_type')['trans_speed'].median()
df['trans_speed'] = df.apply(lambda row: speed_group_median.get(row['trans_type'], df['trans_speed'].median()) if pd.isna(row['trans_speed']) else row['trans_speed'], axis=1)

Next I will save the clean and pre processed data file which will be further used in our analasys and model training.

In [7]:
df.to_csv("../src/data/used_cars_cleaned.csv", index=False, header=True)